# Extraction de parcours de bus OSM en JsonGraph

In [1]:
import requests
import json

In [2]:
lines_metadata = {}
stops_by_lines = {}
stops = []

In [3]:
network = "RTUD"
operator = "RTUD"
overpass_base_url = "http://overpass-api.de/api/interpreter?data="

overpass_query_lines = """
    [out:json];
    (relation["type"="route"]["network"="{}"]["operator"="{}"];)->.a;
    relation(br)["type"="route_master"];
    out meta;""".format(network, operator)

get_lines = requests.get(overpass_base_url + overpass_query_lines)
if not get_lines.status_code == 200 :
    print ("erreur à l'appel d'Overpass pour récupérer les lignes")
    exit
    
lines = get_lines.json()['elements']

In [4]:
routes_id = {}

for a_line in lines :
    lines_metadata[a_line['id']]=a_line['tags']
    routes_id[a_line['id']] = a_line["members"][0]['ref'] #for now, we only inspect one route of each line    

In [5]:
def get_overpass_query_for_stops(route_id):
    return """ [out:json];relation({});(._;>;);out;""".format(route_id)


for a_line_id, a_route_id in routes_id.items() :
    get_route = requests.get(overpass_base_url + get_overpass_query_for_stops(a_route_id))
    if not get_route.status_code == 200:
        break
    all_info = get_route.json()['elements']
    route_info = [elem for elem in all_info if elem['type']=='relation']
    route_members = route_info[0]['members']
    route_stops = [elem for elem in route_members if elem['role']=='platform' and elem['type']=='node']
    stop_list_ids = [elem['ref'] for elem in route_stops]
    stops_by_lines[a_line_id] = stop_list_ids
    stop_info = [elem for elem in all_info if elem["id"] in stop_list_ids]
    stops += stop_info
    

    
print (stops_by_lines)
print (stops)

{6941657: [], 6932554: [4822825493, 4822825492, 4822825491, 4822825490, 4822825486, 4822825484, 4822825482, 4822825481, 4822825480, 4822825479, 4822825478, 4822825628, 4822825477, 4822825475, 4822825474, 4822825473, 4822825471, 4822825470, 4822825467, 4822825466, 2155045951, 4430659470, 4822825476, 4822825495], 6937875: [5300752876, 5300752879, 5300752877, 4822825473, 4822825626, 4822825458, 4822825508, 4822825456], 6937499: [], 7835159: [4822825456, 5300791450, 5300791451, 4822825483, 4822825485, 4822825489, 4822825454, 4822825493, 4822825490, 4822825480]}
[{'id': 4822825456, 'tags': {'bench': 'yes', 'bus': 'yes', 'wheelchair': 'yes', 'name': 'Charles de Gaulle', 'public_transport': 'platform', 'shelter': 'yes', 'highway': 'bus_stop'}, 'lat': 44.0931619, 'lon': 6.2355234, 'type': 'node'}, {'id': 4822825458, 'tags': {'public_transport': 'platform', 'bus': 'yes', 'wheelchair': 'yes', 'highway': 'bus_stop', 'name': 'Rond-Point du 4 Septembre 1970'}, 'lat': 44.0893608, 'lon': 6.2270895, '

In [6]:
nodes = []

for stop in stops:
    stop_name = ""
    if 'name' in stop['tags']:
        stop_name = stop['tags']['name']
    nodes.append({
        "id": stop['id'],
        "lat" : stop['lat'],
        "lon" : stop['lon'],
        "label": stop_name} )

nodes

[{'id': 4822825456,
  'label': 'Charles de Gaulle',
  'lat': 44.0931619,
  'lon': 6.2355234},
 {'id': 4822825458,
  'label': 'Rond-Point du 4 Septembre 1970',
  'lat': 44.0893608,
  'lon': 6.2270895},
 {'id': 4822825473,
  'label': 'Georges Pompidou',
  'lat': 44.085788,
  'lon': 6.215886},
 {'id': 4822825508,
  'label': 'Pierre Gassendi',
  'lat': 44.0903636,
  'lon': 6.2308899},
 {'id': 4822825626,
  'label': 'Les Arcades',
  'lat': 44.0879891,
  'lon': 6.2208534},
 {'id': 5300752876, 'label': 'Le Moulin', 'lat': 44.0803118, 'lon': 6.208642},
 {'id': 5300752877,
  'label': 'Chemin du Marquis',
  'lat': 44.0819127,
  'lon': 6.2153388},
 {'id': 5300752879,
  'label': 'Chemin du Moulin',
  'lat': 44.080952,
  'lon': 6.2114053},
 {'id': 2155045951,
  'label': 'Lycée Pierre-Gilles de Gennes',
  'lat': 44.0727314,
  'lon': 6.1808373},
 {'id': 4430659470,
  'label': 'Digne-les-Bains - Centre Hospitalier',
  'lat': 44.0704794,
  'lon': 6.1779864},
 {'id': 4822825466,
  'label': 'Digne-les-Ba

In [7]:
edges = []

for line in stops_by_lines:
    stops = stops_by_lines[line]

    for i in range(len(stops) -1 ):
        current_edge = {"relation":"bus"}
        current_edge["metadata"] = lines_metadata[line]
        current_edge["source"] = stops[i]
        current_edge["target"] = stops[i+1]
        edges.append(current_edge)

len(edges)
        

39

In [8]:
with open("edges.json", "w") as edges_files :
    json.dump(edges, edges_files, indent=1, sort_keys=True)

with open("nodes.json", "w") as nodes_files :
    json.dump(nodes, nodes_files, indent=0)

In [9]:
stops_by_lines

{6932554: [4822825493,
  4822825492,
  4822825491,
  4822825490,
  4822825486,
  4822825484,
  4822825482,
  4822825481,
  4822825480,
  4822825479,
  4822825478,
  4822825628,
  4822825477,
  4822825475,
  4822825474,
  4822825473,
  4822825471,
  4822825470,
  4822825467,
  4822825466,
  2155045951,
  4430659470,
  4822825476,
  4822825495],
 6937499: [],
 6937875: [5300752876,
  5300752879,
  5300752877,
  4822825473,
  4822825626,
  4822825458,
  4822825508,
  4822825456],
 6941657: [],
 7835159: [4822825456,
  5300791450,
  5300791451,
  4822825483,
  4822825485,
  4822825489,
  4822825454,
  4822825493,
  4822825490,
  4822825480]}